In [2]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy, pickle
from tqdm import tqdm
from scipy import ndimage

In [3]:
# Constants
epsilon = 1e-10

### Supporting Functions

#### Data extraction

In [4]:
def get_input_data(activation_path, trajectory_path):
    with open(trajectory_path, 'rb') as f:
        data_pos = pickle.load(f)

    with open(activation_path, 'rb') as f:
        activations = pickle.load(f)

    #data_pos['x'][0],data_pos['x'][0] = -0.7,-0.7
    x = np.asarray(data_pos['x'])
    y = np.asarray(data_pos['y'])
    positions = np.column_stack((x, y))
    return positions, activations

### Maps

In [5]:
def matlab_style_gauss2D(shape,sigma):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
      h /= sumh
    return h

def occupancy_map_func(pos, reso = 40, win_len = 9):
    '''
    Calculate Occupancy map
    '''
    H1, _, _ = np.histogram2d(pos[:,0], pos[:,1], bins=reso*2)

    gaussian = matlab_style_gauss2D([win_len, win_len], 3.0)
    pos_prob = scipy.signal.convolve2d(gaussian, H1)
    pos_prob = ndimage.rotate(pos_prob, 1*0)
    pos_prob = pos_prob[int(win_len/2):-int(win_len/2), int(win_len/2):-int(win_len/2)]

    pos_prob = (pos_prob)/np.nansum(pos_prob)
    pos_prob[pos_prob == 0] = np.nan
    return pos_prob

def firing_rate_map_place_cells(resp_neurons, positions, occupancy_map,thresh_param = 0.0, title="", ticks_colbar = None, res_param = 40, sub_plot = False):
    '''
    Getting Firing rate map from activation and position data, and occupancy map -> No. of spike counts or activation in occupancy time
    '''
    res = res_param
    mean_resp = np.mean(resp_neurons)
    std_resp = np.std(resp_neurons)
    thresh = mean_resp + (thresh_param * std_resp)
    # thresh = np.max(ot)*thresh_param
    firr = np.nonzero(resp_neurons>thresh)
    firposgrid = positions[firr[0], :2]
    #firr = list(firr[0])
    x = np.arange(-1, 1, 1/res)
    y = np.arange(-1, 1, 1/res)
    fx,fy = np.meshgrid(x, y)
    firingmap = np.zeros(fx.shape)
    win_len = 9
    firingvalue = resp_neurons[firr]
    for ii in range(len(firposgrid)):
        q1 = np.argmin(abs(firposgrid[ii,0] - fx[1,:]))
        q2 = np.argmin(abs(firposgrid[ii,1] - fy[:,1]))
        firingmap[q1,q2] = max(firingvalue[ii], firingmap[q1,q2])
    # firingmap = firingmap/max(np.max(firingmap),1)
    occupancy_map += epsilon
    firingmap = firingmap/occupancy_map
    gaussian = matlab_style_gauss2D([9, 9], 3)
    spikes_smooth = scipy.signal.convolve2d(gaussian, firingmap)
    rotated_img = ndimage.rotate(spikes_smooth, 1*90)
    rotated_img = rotated_img[int(win_len/2):-int(win_len/2), int(win_len/2):-int(win_len/2)]
    if not sub_plot:
      plt.imshow(rotated_img, origin= 'upper', cmap = 'jet')
      plt.title(title)
      plt.colorbar(ticks = ticks_colbar)
      # plt.clim(0,1)
      ax=plt.gca()                            # get the axis
      ax.set_ylim(ax.get_ylim()[::-1])        # invert the axis
      # ax.set_xlim(ax.get_xlim()[::-1])        # invert the axis
      plt.show()
      # ax.xaxis.tick_bottom()                     # and move the X-Axis
      # ax.set_yticklabels([])
      # ax.set_xticklabels([])
    return rotated_img

### Scores

In [6]:
def sparsity_func(pos_prob, firing_rate):
    '''
    Calculating sparsity for a given firing rate map
    input
    pos_prob: probability matrix for position - from occupancy map
    firing_rate: rate map

    output: sparsity from the paper
    '''
    pos_prob2 = np.copy(pos_prob)
    pos_prob2 = pos_prob2/(np.nansum(pos_prob2) + epsilon)

    avg_rate = np.nansum(np.ravel(firing_rate * pos_prob2))
    numerator = avg_rate ** 2
    denominator = np.nansum(np.multiply(pos_prob2, firing_rate**2)) + epsilon
    return numerator/denominator

In [7]:
def _inf_rate(rate_map, px):
    '''Calculating information rate.
    rate_map: firing rate map
    px: occupancy map
    '''
    tmp_map = np.ma.array(rate_map, copy=True)
    tmp_map[np.isnan(tmp_map)] = 0
    avg_rate = np.sum(np.ravel(tmp_map * px))

    #return (np.nansum(np.ravel(tmp_map * np.log2(tmp_map/avg_rate) * px)),
    #        avg_rate) 

    return (np.nansum(np.ravel(tmp_map * np.log2(np.maximum(tmp_map/avg_rate, epsilon)) * px)),
            avg_rate) 

In [8]:
def get_scores(positions, activations):
    '''
    Calculating sparsity,  spatial information rate and spatial score
    '''

    global lim, reso
    firing_maps = []
    sparsity_scores = []
    spatial_info = []
    occupancy_map = occupancy_map_func(positions)
    resp_neurons = activations.T

    num = 0
    for j in range(int(np.divide(len(resp_neurons), resp_neurons.shape[0]))):
        for i in range(resp_neurons.shape[0]):

          fr_map = firing_rate_map_place_cells(resp_neurons[i+num], positions, occupancy_map = occupancy_map, thresh_param=lim,
                                    res_param = reso, sub_plot = True)

          # Calculating sparsity
          sparsity_cell = sparsity_func(occupancy_map, fr_map)
          if np.ma.is_masked(sparsity_cell):
              sparsity_cell = 0  # Replace masked value with 0 or any other default value you prefer
          else:
              sparsity_cell = round(sparsity_cell,2)
          sparsity_scores.append(sparsity_cell)


          sp_info_rate, avg_rate = _inf_rate(fr_map, occupancy_map)

          # Calculating spatial score
          spatial_score = sp_info_rate*0.1/avg_rate
          if np.ma.is_masked(spatial_score):
              spatial_score = 0  # Replace masked value with 0 or any other default value you prefer
          else:
              spatial_score = round(spatial_score,2)
          spatial_info.append(spatial_score)
        num = num + resp_neurons.shape[0]

    return spatial_info, sparsity_scores

### Identifying place cells

In [9]:
def identify_place_cells(spatial_scores, sparsity, positions, activations):
    selected_neus_spars = np.where(np.asarray(sparsity) < 0.1)[0]
    selected_neus_spainfo = np.where(np.asarray(spatial_scores) > 0.3)[0]
    selected_neus = set(selected_neus_spars) & set(selected_neus_spainfo)
    [percentiles_spatial_info, percentiles_sparsity] =  (selected_neus, positions, activations)
    desired_spatial_info = [spatial_scores[i] for i in selected_neus]
    desired_sparsity_score = [sparsity[i] for i in selected_neus]
    comparison_result_spatial_info = [1 if a > b else 0 for a, b in zip(desired_spatial_info, percentiles_spatial_info)]
    comparison_result_sparsity_score = [1 if a < b else 0 for a, b in zip(desired_sparsity_score, percentiles_sparsity)]

    spatial_and_sparisty = comparison_result_sparsity_score and comparison_result_spatial_info

    place_cells = [neuron for neuron, binary_value in zip(selected_neus, spatial_and_sparisty) if binary_value == 1]
    place_cells = [element + 1   for element in place_cells]

    return place_cells

In [10]:
def get_scores_by_place_cells(place_cells, spatial_scores, sparsity):
    place_cells_spatial_s = [spatial[i-1] for i in place_cells]
    place_cells_sparsity = [sparsity[i-1] for i in place_cells]
    pc_dict = {}

    for i in range(len(place_cells)):

        indices_index.append(place_cells[i])
        si.append(place_cells_sparsity[i])
        ss.append(place_cells_spatial[i])
        pc_dict= {
            'place_cells_indices': indices_index,
            'spatial_scores':place_cells_sparsity,
            'sparsity_scores':place_cells_spatial_s
        }


    return pc_dict

#### Download DONN activations & extract corresponding positions

In [12]:
# Get input data
activation_path = '0_input_data/activations_z3_traj_5.pkl'
trajectory_path = '0_input_data/traj_5_data.pkl'
reso = 40
lim = 1.5

positions, activations = get_input_data(activation_path, trajectory_path)

In [13]:
print(positions.shape)
print(activations.shape)

(1000, 2)
(1000, 100)


### Calculate spatial score and sparsity

In [14]:
spatial_scores, sparsity = get_scores(positions, activations)

/tmp/ipykernel_3748961/1409061526.py:51: ComplexWarning: Casting complex values to real discards the imaginary part
  firingmap[q1,q2] = max(firingvalue[ii], firingmap[q1,q2])


### Identify place cells obeying following conditions

Sparsity < 0.1

Spatial score > 0.3 bits/spike

In [15]:
place_cells = identify_place_cells(spatial_scores, sparsity, positions, activations)

ValueError: too many values to unpack (expected 2)

In [16]:
place_cell_dict = get_scores_by_place_cells(place_cells, spatial_scores, sparsity)

NameError: name 'place_cells' is not defined

### Store all identified PCs and corresponding scores

In [ ]:
save_path = f'/{model_name}_{simulation_details}_{layer}_DONN_place_cell_dict.pkl'
with open(save_path, 'wb') as f:
    pickle.dump(place_cell_dict, f)